# 3) Model Training & Evaluation — Calibrated Boosting & Automated Search


We aim for a model that **ranks customers by likelihood** to say yes, with **reliable probabilities** for planning. We leverage modern **gradient boosting (HGB)**, **calibrate** probabilities, and use **automated hyperparameter search**.

In [1]:

# Common setup
import os,warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [3]:

import joblib, json
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance, partial_dependence

DATA_PATH = "./data/processed.csv"
PREPROC_PATH = "./models/preprocessor.joblib"
FEATURE_META_PATH = "./models/feature_meta.json"
SPLIT_PATH = "./outputs/split_indices.json"
BEST_MODEL_PATH = "./models/best_model.joblib"
BEST_CALIBRATED_PATH = "./models/best_model_calibrated.joblib"
METRICS_PATH = "./outputs/metrics.json"

df = pd.read_csv(DATA_PATH)
with open(SPLIT_PATH) as f:
    idx = json.load(f)
with open(FEATURE_META_PATH) as f:
    meta = json.load(f)
preprocess = joblib.load(PREPROC_PATH)

y_all = (df["y"].astype(str).str.lower() == "yes").astype(int)
X_all = df.drop(columns=["y"])
X_train, y_train = X_all.loc[idx["train_idx"]], y_all.loc[idx["train_idx"]]
X_valid, y_valid = X_all.loc[idx["valid_idx"]], y_all.loc[idx["valid_idx"]]

# Transform
Xtr = preprocess.transform(X_train)
Xva = preprocess.transform(X_valid)

# Baseline: Logistic Regression (calibrated)
logreg = LogisticRegression(max_iter=200, class_weight="balanced")
logreg.fit(Xtr, y_train)
logreg_cal = CalibratedClassifierCV(logreg, method="isotonic", cv=5)
logreg_cal.fit(Xtr, y_train)

proba_lr = logreg_cal.predict_proba(Xva)[:,1]
metrics_lr = {
    "roc_auc": float(roc_auc_score(y_valid, proba_lr)),
    "pr_auc": float(average_precision_score(y_valid, proba_lr)),
    "brier": float(brier_score_loss(y_valid, proba_lr))
}
print("LogReg (calibrated) metrics:", metrics_lr)

# Advanced: HistGradientBoosting with hyperparameter search
hgb = HistGradientBoostingClassifier(random_state=RANDOM_STATE, max_bins=255, class_weight=None)
param_dist = {
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "max_depth": [None, 3, 5, 7],
    "min_samples_leaf": [10, 20, 50, 100],
    "l2_regularization": [0.0, 0.1, 0.5, 1.0]
}

try:
    search = HalvingRandomSearchCV(
        estimator=hgb, param_distributions=param_dist, factor=3, resource="max_iter",
        max_resources=300, min_resources=50, random_state=RANDOM_STATE, n_candidates="exhaust",
        scoring="average_precision", cv=3, verbose=1
    )
except Exception as e:
    # Fallback to RandomizedSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    search = RandomizedSearchCV(
        estimator=hgb, param_distributions=param_dist, n_iter=20, random_state=RANDOM_STATE,
        scoring="average_precision", cv=3, verbose=1
    )

search.fit(Xtr, y_train)
best_hgb = search.best_estimator_
print("Best HGB params:", search.best_params_)

# Calibrate best model
cal_hgb = CalibratedClassifierCV(best_hgb, method="isotonic", cv=5)
cal_hgb.fit(Xtr, y_train)

proba_hgb = cal_hgb.predict_proba(Xva)[:,1]
metrics_hgb = {
    "roc_auc": float(roc_auc_score(y_valid, proba_hgb)),
    "pr_auc": float(average_precision_score(y_valid, proba_hgb)),
    "brier": float(brier_score_loss(y_valid, proba_hgb)),
}

print("HGB (calibrated) metrics:", metrics_hgb)

# Choose best by PR-AUC (ranking quality for imbalanced data)
best_name = "HGB_calibrated" if metrics_hgb["pr_auc"] >= metrics_lr["pr_auc"] else "LogReg_calibrated"
best_model = cal_hgb if best_name=="HGB_calibrated" else logreg_cal

import joblib
joblib.dump(best_model, BEST_CALIBRATED_PATH)
joblib.dump(best_model.base_estimator if hasattr(best_model, "base_estimator") else best_model, BEST_MODEL_PATH)

with open(METRICS_PATH, "w") as f:
    json.dump({"best_model": best_name, "logreg": metrics_lr, "hgb": metrics_hgb}, f, indent=2)

print("Saved best calibrated model ->", BEST_CALIBRATED_PATH)
print("Metrics ->", METRICS_PATH)

# --- Tests: model should beat a naive baseline ---
pos_rate = y_valid.mean()
pr_auc_naive = pos_rate  # PR-AUC of a random baseline = positive prevalence
assert max(metrics_lr["pr_auc"], metrics_hgb["pr_auc"]) > pr_auc_naive, "Model didn't outperform naive baseline PR-AUC."
print("✅ Baseline-beating test passed.")


LogReg (calibrated) metrics: {'roc_auc': 0.9428311034718617, 'pr_auc': 0.6194791590941963, 'brier': 0.05812632497636531}
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 50
max_resources_: 300
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 6
n_resources: 50
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 1
n_candidates: 2
n_resources: 150
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best HGB params: {'min_samples_leaf': 20, 'max_depth': 5, 'learning_rate': 0.1, 'l2_regularization': 0.5, 'max_iter': 150}
HGB (calibrated) metrics: {'roc_auc': 0.9547829496674373, 'pr_auc': 0.6990778590803219, 'brier': 0.0524683337468139}
Saved best calibrated model -> ./models/best_model_calibrated.joblib
Metrics -> ./outputs/metrics.json
✅ Baseline-beating test passed.


### Evaluation Visuals (Plotly) — ROC, PR, Calibration, Gains/Lift, Feature Insights

In [4]:

import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.calibration import calibration_curve

# Select proba
proba = proba_hgb if best_name=="HGB_calibrated" else proba_lr

# ROC
fpr, tpr, _ = roc_curve(y_valid, proba)
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode="lines", name="ROC"))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines", name="Chance", line=dict(dash="dash")))
fig.update_layout(title="ROC Curve", xaxis_title="FPR", yaxis_title="TPR", template="plotly_white")
fig.show()

# PR
prec, rec, _ = precision_recall_curve(y_valid, proba)
fig = go.Figure()
fig.add_trace(go.Scatter(x=rec, y=prec, mode="lines", name="PR"))
fig.add_trace(go.Scatter(x=[0,1], y=[y_valid.mean()]*2, mode="lines", name="Baseline", line=dict(dash="dash")))
fig.update_layout(title="Precision-Recall Curve", xaxis_title="Recall", yaxis_title="Precision", template="plotly_white")
fig.show()

# Calibration (Reliability)
prob_true, prob_pred = calibration_curve(y_valid, proba, n_bins=10, strategy="quantile")
fig = go.Figure()
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode="lines+markers", name="Calibration"))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines", name="Perfect", line=dict(dash="dash")))
fig.update_layout(title="Calibration Curve", xaxis_title="Predicted Probability", yaxis_title="Observed Frequency", template="plotly_white")
fig.show()

# Cumulative Gains & Lift
order = np.argsort(-proba)
y_sorted = y_valid.iloc[order].reset_index(drop=True)
cum_resp = y_sorted.cumsum()
pct_customers = np.arange(1, len(y_sorted)+1) / len(y_sorted)
pct_responders = cum_resp / y_valid.sum()
lift = (pct_responders / pct_customers)

fig = go.Figure()
fig.add_trace(go.Scatter(x=pct_customers, y=pct_responders, mode="lines", name="Cumulative Gains"))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines", name="Baseline", line=dict(dash="dash")))
fig.update_layout(title="Cumulative Gains", xaxis_title="% Customers Called", yaxis_title="% Responders Captured", template="plotly_white")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=pct_customers, y=lift, mode="lines", name="Lift"))
fig.update_layout(title="Lift Curve", xaxis_title="% Customers Called", yaxis_title="Lift vs Random", template="plotly_white")
fig.show()


In [5]:

# Permutation importance (top 20 features), with feature name reconstruction
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse

enc = None
for name, trans, cols in joblib.load(PREPROC_PATH).transformers_:
    if name == "cat":
        enc = trans
        cat_cols = cols
    elif name == "num":
        num_cols = cols

# Build feature names
feat_names = []
if num_cols:
    feat_names.extend(num_cols)
if enc is not None:
    cats = enc.categories_
    for col, categories in zip(cat_cols, cats):
        feat_names.extend([f"{col}={val}" for val in categories])

# Compute permutation importances on validation
pi = permutation_importance(best_model, Xva, y_valid, n_repeats=5, random_state=RANDOM_STATE, n_jobs=-1)
importances = pd.DataFrame({
    "feature": feat_names,
    "importance_mean": pi.importances_mean,
    "importance_std": pi.importances_std
}).sort_values("importance_mean", ascending=False).head(20)

fig = px.bar(importances.sort_values("importance_mean"),
             x="importance_mean", y="feature", error_x="importance_std", orientation="h",
             title="Permutation Importance — Top 20", labels={"importance_mean":"Mean ΔScore", "feature":"Feature"})
fig.update_layout(template="plotly_white", height=700)
fig.show()
